In [1]:
import elite_ai_ml_bckend

Package Dependencies That Can Cause Error
 'sentence-transformers', 'sumy'



In [2]:
!pip install sentence-transformers
!pip install sumy

The system cannot find the path specified.


The system cannot find the path specified.


In [3]:
from elite_ai_ml_bckend.elite_ai_content_summarisation import *
from elite_ai_ml_bckend.alternative_summarizations import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T.PRANEETH\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T.PRANEETH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
##CODE TO RECEIVE MAILS
import email
import imaplib
import datetime
import smtplib


def extract_textin_mails(email_id, password, folder="Inbox", display=False, mail_count=100, **kwargs):
    EMAIL = email_id
    PASSWORD = password
    SERVER = 'imap.gmail.com'

    # connect to the server and go to its inbox
    mail = imaplib.IMAP4_SSL(SERVER)
    mail.login(EMAIL, PASSWORD)
    # we choose the inbox but you can select others
    mail.select('"{0}"'.format(folder))

 
    # we'll search using the ALL criteria to retrieve
    # every message inside the inbox
    # it will return with its status and a list of ids
    search_cue = '('
    for k,v in kwargs.items():
        search_cue += k+" "
        search_cue += '"{0}"'.format(v) + " "
    search_cue = search_cue[:len(search_cue)-1]
    search_cue += ')'
            
    try:
        status, data = mail.search(None, search_cue)
    except:
        print("SEARCH_CUE ERROR")
        return

    # the list returned is a list of bytes separated
    # by white spaces on this format: [b'1 2 3', b'4 5 6']
    # so, to separate it first we create an empty list
    mail_ids = []
    # then we go through the list splitting its blocks
    # of bytes and appending to the mail_ids list
    for block in data:
        # the split function called without parameter
        # transforms the text or bytes into a list using
        # as separator the white spaces:
        # b'1 2 3'.split() => [b'1', b'2', b'3']
        mail_ids += block.split()
    mail_ids = mail_ids[:mail_count]
    # now for every id we'll fetch the email
    # to extract its content
    mail_contents=[]
    mail_subjects = []
    for i in mail_ids:
        # the fetch function fetch the email given its id
        # and format that you want the message to be
        status, data = mail.fetch(i, '(RFC822)')

        # the content data at the '(RFC822)' format comes on
        # a list with a tuple with header, content, and the closing
        # byte b')'
        for response_part in data:
            # so if its a tuple...
            if isinstance(response_part, tuple):
                # we go for the content at its second element
                # skipping the header at the first and the closing
                # at the third
                message = email.message_from_bytes(response_part[1])

                # with the content we can extract the info about
                # who sent the message and its subject
                mail_from = message['from']
                mail_subject = message['subject']
                
                
                    
                # then for the text we have a little more work to do
                # because it can be in plain text or multipart
                # if its not plain text we need to separate the message
                # from its annexes to get the text
                if message.is_multipart():
                    mail_content = ''

                    # on multipart we have the text message and
                    # another things like annex, and html version
                    # of the message, in that case we loop through
                    # the email payload
                    for part in message.get_payload():
                        # if the content type is text/plain
                        # we extract it
                        if part.get_content_type() == 'text/plain':
                            mail_content += part.get_payload()
                        else:
                            
                            for subpart in part.get_payload():
                                
                                try:
                                    if subpart.get_content_type() != 'text/plain':
                                        for subsubpart in subpart.get_payload():
                                            if subsubpart.get_content_type() == 'text/plain':
                                                mail_content += subsubpart.get_payload() #== "text/plain":
                                    else:
                                        mail_content += subpart.get_payload()
                                except :
                                    pass
                                    
                            
                else:
                    # if the message isn't multipart, just extract it
                    mail_content = message.get_payload()

                # and then let's show its result
                if display == True:
                    print(f'From: {mail_from}')
                    print(f'Subject: {mail_subject}')
                    print(f'Content: {mail_content}')
                mail_contents.append(mail_content)
                mail_subjects.append(mail_subject)
    return mail_contents, mail_subjects






def send_summaryin_mail(summary_file=None, summary_text=None, subject=None, sender_mail_id=None, sender_mail_password=None, receiver_mail_id=None):
    # Python code to illustrate Sending mail from
# your Gmail account
    
    if summary_file != None:
        summ_file = open(summary_file, 'r')
        summary = summ_file.read()
    else:
        summary = summary_text
# creates SMTP session
    s = smtplib.SMTP('smtp.gmail.com', 587)

    # start TLS for security
    s.starttls()

    # Authentication
    s.login(sender_mail_id, sender_mail_password)

    # message to be sent
    

    message = "\n\n\n\tElite Summary of your " + subject.upper() + " Mail\n\n\n" + summary

    # sending the mail
    s.sendmail(sender_mail_id, receiver_mail_id, message)

    # terminating the session
    s.quit()
    
    return "Successfully Sent Mail"


In [80]:
mail_contents, mail_subjects = extract_textin_mails(email_id="praneeth_thota@srmap.edu.in", password="7ujm^%ty", folder="Inbox", display=False, ON='06-Sep-2021', mail_count=3)
len(mail_contents)

3

In [82]:
summaries_and_subjects = []
for mi in range(len(mail_contents)):
    if len(mail_contents[mi]) != 0:
        try:
            result = execute_summarisation(mail_contents[mi], summary_ratio=0.5)
            summaries_and_subjects.append((result['EnsembledSummary'], mail_subjects[mi]))
        except:
            print("Error With Summarisation")


~DONE~
Error With Summarisation
~DONE~


In [83]:
for si in range(len(summaries_and_subjects)):
    status = send_summaryin_mail(summary_text=summaries_and_subjects[si][0], subject=summaries_and_subjects[si][1], sender_mail_id='praneeth_thota@srmap.edu.in', sender_mail_password='7ujm^%ty',receiver_mail_id='praneeth_thota@srmap.edu.in')
    print("Status of Mail", si+1, ":", status)

Status of Mail 1 : Successfully Sent Mail
Status of Mail 2 : Successfully Sent Mail


In [13]:
print(result['EnsembledSummary'])

Dear All, Good morning and Happy Teachers' Day!
The auspicious celebration of "Teachers' Day" originated to commemorate the birthday of Dr Sarvepalli Radhakrishnan, former President of India, by acknowledging the contributions of teachers- the best minds of the country.
We must salute Dr Radhakrishnan and remember his words of wisdom that "the end-product of education should be a free creative man, who can battle against historical circumstances and adversities of nature."



In [1]:
#mail.lsub(directory='""', pattern='*')

Dear All,

Good morning and Happy Teachers' Day!

The auspicious celebration of *"Teachers' Day"* originated to commemorate
the birthday of Dr Sarvepalli Radhakrishnan, former President of India, by
acknowledging the contributions of teachers- the best minds of the country.
Sarvepalli Radhakrishnan was a highly-learned man, philosopher, prolific
statesman and a remarkable teacher himself.

We must salute Dr Radhakrishnan and remember his words of wisdom that *"the
end-product of education should be a free creative man, who can battle
against historical circumstances and adversities of nature."*

[image: teachers-day.jpg]
--=20

Thanks and Regards,



*Prof. V S Rao*

*Vice Chancellor*

SRM University, AP

Andhra Pradesh =E2=80=93 522502

Email : vc@srmap.edu.in

